In [ ]:
!pip install transformers tokenizers datasets

In [4]:
input_filename = "naslovi.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'distilgpt2'
new_model_name = pretrained_model_name + '_1'

In [6]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="naslovi-tokenizer.json")
tokenizer.pad_token = "<pad>"


In [5]:
from datasets import load_dataset
# datasets = load_dataset('text', data_files=dataset_filename, split="train[90%],validate")
eval, train = load_dataset('text', data_files=dataset_filename, split=['train[:10%]', 'train[10%:]'])
def prepare_for_trainer(example):
  tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=20)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized
tokenized_train = train.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])
tokenized_eval = eval.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])


Using custom data configuration default-9aeaa2985c497c83


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9aeaa2985c497c83/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.
  

In [6]:
from transformers import Trainer, AutoConfig, AutoModelForCausalLM, TrainingArguments

config = AutoConfig.from_pretrained(pretrained_model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
# model.to('cuda:0')
training_args = TrainingArguments(
    "new_model_name",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # prediction_loss_only=True,
    tokenizer=tokenizer,
)


In [7]:
import logging
logger = logging.getLogger(__name__)
logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [8]:
train_result = trainer.train()
trainer.save_model('./modeli/' + new_model_name)

***** Running training *****
  Num examples = 231643
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 86868


Epoch,Training Loss,Validation Loss
1,5.702300,6.069186
2,5.450100,5.766161
3,5.334600,5.681875


***** Running Evaluation *****
  Num examples = 25738
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-28956
Configuration saved in test-clm/checkpoint-28956/config.json
Model weights saved in test-clm/checkpoint-28956/pytorch_model.bin
tokenizer config file saved in test-clm/checkpoint-28956/tokenizer_config.json
Special tokens file saved in test-clm/checkpoint-28956/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 25738
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-57912
Configuration saved in test-clm/checkpoint-57912/config.json
Model weights saved in test-clm/checkpoint-57912/pytorch_model.bin
tokenizer config file saved in test-clm/checkpoint-57912/tokenizer_config.json
Special tokens file saved in test-clm/checkpoint-57912/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 25738
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-86868
Configuration saved in test-clm/checkpoint-86868/config.

In [7]:
import numpy as np
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('./modeli/' + new_model_name)
model.config.pad_token_id = 1

for temp in np.linspace(0.2, 2, 10):
    print()
    print("###### Temperature: " + str(temp))
    for i in range(0, 5):
        out = model.generate(do_sample=True, temperature=temp)[0]
        print(tokenizer.decode(out, skip_special_tokens=True))


###### Temperature: 0.2
U BiH se u Srbiji je bio u Srbiji, a onda je to je to usledio i to
U BiH se ne zna da li će biti biti u Srbiji
U BiH u Srbiji se ne zna da li će biti i dalje ne može biti na Kosovu i
U BiH i dalje u Srbiji se sa Vučićem, a onda je usledio je to usledio i to
U BiH se ne može da se ne može biti na koronu, a onda je to

###### Temperature: 0.4
Da li će tovati da bude biti, ali i dalje ne može da bude to?
U Rusiji dve osobe povređene dobili dve osobe povređene
Orska je da se ne sme, ali je da je to što pre nego što je to
Na snagu u BiH i dalje na poklon nad 10
Netanijahu je u Srbiji, a onda je bio u Srbiji je bila u Srbiji, a onda

###### Temperature: 0.6000000000000001
Vučić  U Srbiji je bio za ubistvo, ali i do 20. godine ne sme da se
Vučić  Zarasata za vikend, na ovim mestu da se za razvoj i koro
U Srpskoj vivinju i Apitajomn i na severu
U stanu i dalje je da je prvi do kraja godine na korona virus
U jednom danu slučajeva korone na koronu pozitivno još uvek oca,